# OpenSearch 한글 형태소 분석기 통한 키워드 검색 
>이 노트북은 SageMaker Studio* Data Science 3.0 kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.



여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. KoSIMCSERoberta 를 세이지 메이커 엔드포인트로 사용하신다면 아래의 선수 조건을 확인하세요.

#### 선수조건 (KoSIMCSERoberta 사용시)
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)
- [OpenSearch Query Description (한글)](https://esbook.kimjmin.net/05-search)


# 1. 환경 세팅

In [6]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

## Bedrock Client 생성
### 선수 지식
아래의 노트북을 먼저 실행해서, Bedrock 에 접근 가능하게 합니다.
- amazon-bedrock-workshop-webinar-kr/00_Setup/setup.ipynb

In [7]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## Embedding 모델 선택

In [8]:
from utils.rag import get_embedding_model
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [9]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 경우,  endpoint_name 을 꼭 넣어 주세요.

In [10]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 2. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [11]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
...,...,...,...,...,...
87,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
88,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
89,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
90,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [12]:
os.makedirs("data", exist_ok=True)

In [13]:
def preprocess_data(df):

    ldf = df.copy()
    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader 로 문서 로딩

In [20]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, SpacyTextSplitter

In [21]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)
import time
documents_fsi = loader.load()


## 메타 데이터 생성
- 기존 Category 컬럼을 ask 로 변경 합니다.
- 컬럼의 type, source 는 metadata 로 생성하고, 내용에서는 삭제 합니다.
- 타임스탬프 및 임베딩 모델의 엔드포인트 이름을 metadata 로 추가 합니다.

In [22]:
import time
documents_fsi = loader.load()

def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):

        #print ("previous:", doc)
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        split_content = doc.page_content.split("type: ")
        content = split_content[0]        
        metadata = split_content[1]                
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        doc.metadata['timestamp'] = time.time()

create_metadata(documents_fsi)

In [23]:
print (len(documents_fsi))
print (documents_fsi[0])

92
page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n' metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1697030980.834936}


## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. <BR>
또한 임베딩 모델에는 입력 토큰 길이는 KoSimCSERobert는 512개, titanEmbedding(8,912개) 토큰으로 제한되어 있습니다. <BR>
이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 500자가 겹치는 약 92자의 청크를 생성합니다.

In [24]:
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 50
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


In [27]:
docs

[Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n\n\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n\n\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1697030980.834936}),
 Document(page_content='ask: 아마존 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \r\n\n\n아마존 공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\r\n\n\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\r\n\r\n\n\n아마존 금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\r\n\n\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)\r\n※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS, SMS, 마이인포앱 

# 3. OpenSearch Client 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
### 아래 셀에 다음의 정보가 입력이 되어야 합니다.
```
opensearch_domain_endpoint = "<Type Domain Endpoint>"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

```


In [15]:
from utils.opensearch import opensearch_utils

In [60]:
aws_region = 'us-east-1'

os.environ["OpenSearch_UserName"] = "<Type UserName>"
os.environ["OpenSearch_UserPassword"] = "<Type Password>"

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

opensearch_domain_endpoint = "<Type your domain endpoint>"

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)


# 4. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [50]:
index_name = "genai-demo-index-v1"
index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
    opensearch_utils.delete_index(os_client, index_name)
else:
    print("Index does not exist")

index_name=genai-demo-index-v1, exists=True

Deleting index:
{'acknowledged': True}


## 인덱스 생성

In [51]:
from langchain.vectorstores import OpenSearchVectorSearch

In [52]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

CPU times: user 202 ms, sys: 21 ms, total: 223 ms
Wall time: 9.23 s


## 인덱스 확인

In [53]:
from utils.rag import to_pickle

In [54]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)
to_pickle(index_info, "../utils/tokenizer-index-info-template.pkl")

{'genai-demo-index-v1': {'aliases': {},
                         'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                 'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                   'type': 'keyword'}},
                                                                                            'type': 'text'},
                                                                                 'timestamp': {'type': 'float'},
                                                                                 'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                 'type': 'keyword'}},
                                                                                          't

## 은전한잎 형태소 분석기 (seunjeon_tokenizer) 사용하기
- 영어권의 문자들과 다르게 한글, 일본어, 중국어 등은 단순한 공백만으로는 좋은 검색 결과를 얻기 힘듭니다.
- 출시하고라는 단어가 들어간 문서를 출시하고라는 정확히 같은 단어만으로 검색할 수 있다면 답답하겠죠?
- 출시하고라는 단어를 출시, 출시하고 등 다양하게 검색하기 위해서는 형태소 분석기가 필요합니다.
- https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming#

### 인덱싱 수정하기 (형태소 분석시 사용 enablement)

In [55]:
new_index_name = f'{index_name}-with-tokenizer'
new_index_name

'genai-demo-index-v1-with-tokenizer'

In [56]:
# Setting for "은전한잎" Tokenizer (변경 없음)
index_info[index_name]["settings"]["analysis"] = {
    "tokenizer": {
        "seunjeon": {
            "type": "seunjeon_tokenizer"
        }
    },
    "analyzer": {
        "my_analyzer": {
            "type": "custom",
            "tokenizer": "seunjeon"
        }
    }
}

# Setting for Columns to be adapted by Tokenizer (tokenizer가 적용될 컬럼에 맞춰서 수정)
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]["mappings"]["properties"]["text"]["search_analyzer"] = "my_analyzer"

# Setting for vector index column (변경 없음)
index_info[index_name]["settings"]["index"] = {
    "number_of_shards": "5",
    "knn.algo_param": {"ef_search": "512"},
    "knn": "true",
    "number_of_replicas": "2"
}
del index_info[index_name]["aliases"]
new_index_info = index_info[index_name]

In [59]:
pprint(new_index_info)

{'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                         'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                           'type': 'keyword'}},
                                                                    'type': 'text'},
                                                         'timestamp': {'type': 'float'},
                                                         'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                         'type': 'keyword'}},
                                                                  'type': 'text'}}},
                             'text': {'analyzer': 'my_analyzer',
                                      'fields': {'keyword': {'ignore_above': 256,
                                                             'type':

### 형태소 분석기용 인덱서 생성

In [61]:
index_exists = opensearch_utils.check_if_index_exists(os_client, new_index_name)
if index_exists:
    opensearch_utils.delete_index(os_client, new_index_name)
else:
    print("Index does not exist")

index_name=genai-demo-index-v1-with-tokenizer, exists=False
Index does not exist


In [62]:
opensearch_utils.create_index(
    os_client,
    index_name=new_index_name,
    index_body=new_index_info
)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-demo-index-v1-with-tokenizer'}


### Re-indexing

In [63]:
_reindex = {
    "source": {"index": index_name},
    "dest": {"index": new_index_name}
}
print("_reindex: \n", _reindex)

_reindex: 
 {'source': {'index': 'genai-demo-index-v1'}, 'dest': {'index': 'genai-demo-index-v1-with-tokenizer'}}


In [64]:
os_client.reindex(_reindex)

{'took': 583,
 'timed_out': False,
 'total': 92,
 'updated': 0,
 'created': 92,
 'deleted': 0,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

# 6. 키워드 검색

### 'Text" 에 "약관, 뱅킹" 단어를 검색합니다.

* without tokenizer (index_name)

In [65]:
query = "뱅킹"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '뱅킹', 'minimum_should_match': '0%', 'operator': 'or'}}}], 'filter': []}}}
There is no response


* with tokenizer (new_index_name)

In [66]:
query = "뱅킹"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '뱅킹', 'minimum_should_match': '0%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  10
# of display: 3
---------------------
_id in index:  42fea4d5-39eb-4269-946d-a10f73d90e75
1.3526084
ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)
{'source': '신한은행', 'row': 39, 'type': '인터넷뱅킹', 'timestamp': 1696832766.291876}
---------------------
_id in index:  f9e647ee-82c8-422a-a197-44a7c424468c
1.2793157
ask: 인터넷뱅킹 송금 수수료는 얼마죠?
Information: 인터넷뱅킹(bank.shinhan.com) 송금 수수료는 우측 퀵메뉴 이용안내 수수료/이용시간안내 인터넷뱅킹 이용수수료 메뉴에서 확인 가능합니다.
{'source': '신한은행', 'row': 86, 'type': '인터넷뱅킹', 'tim

## 형태소 분석 결과 확인
"약관" 또는 "뱅킹" 확인 <BR>
#### [중요]:  doc_id: 위의 문서 인덱스 정보 확인 후 수정

In [67]:
doc_id = "de0ed395-c0fc-423f-9759-5736cc35541d"

* without tokenizer

In [68]:
os_client.termvectors(index=index_name, id=doc_id, fields='text')

{'_index': 'genai-demo-index-v1',
 '_id': 'de0ed395-c0fc-423f-9759-5736cc35541d',
 '_version': 1,
 'found': True,
 'took': 1,
 'term_vectors': {'text': {'field_statistics': {'sum_doc_freq': 829,
    'doc_count': 18,
    'sum_ttf': 1041},
   'terms': {'ahnlab': {'term_freq': 1,
     'tokens': [{'position': 16, 'start_offset': 88, 'end_offset': 94}]},
    'ask': {'term_freq': 1,
     'tokens': [{'position': 0, 'start_offset': 0, 'end_offset': 3}]},
    'astx': {'term_freq': 3,
     'tokens': [{'position': 5, 'start_offset': 29, 'end_offset': 33},
      {'position': 15, 'start_offset': 81, 'end_offset': 85},
      {'position': 94, 'start_offset': 483, 'end_offset': 487}]},
    'astx프로그램을': {'term_freq': 1,
     'tokens': [{'position': 64, 'start_offset': 329, 'end_offset': 338}]},
    'information': {'term_freq': 1,
     'tokens': [{'position': 9, 'start_offset': 44, 'end_offset': 55}]},
    'pc부팅시': {'term_freq': 3,
     'tokens': [{'position': 26, 'start_offset': 147, 'end_offset': 152}

* with tokenizer

In [69]:
os_client.termvectors(index=new_index_name, id=doc_id, fields='text')

{'_index': 'genai-demo-index-v1-with-tokenizer',
 '_id': 'de0ed395-c0fc-423f-9759-5736cc35541d',
 '_version': 1,
 'found': True,
 'took': 22,
 'term_vectors': {'text': {'field_statistics': {'sum_doc_freq': 1268,
    'doc_count': 18,
    'sum_ttf': 1996},
   'terms': {'ahnlab/SL': {'term_freq': 1,
     'tokens': [{'position': 21, 'start_offset': 88, 'end_offset': 94}]},
    'ask/SL': {'term_freq': 1,
     'tokens': [{'position': 0, 'start_offset': 0, 'end_offset': 3}]},
    'astx/SL': {'term_freq': 4,
     'tokens': [{'position': 8, 'start_offset': 29, 'end_offset': 33},
      {'position': 20, 'start_offset': 81, 'end_offset': 85},
      {'position': 92, 'start_offset': 329, 'end_offset': 333},
      {'position': 138, 'start_offset': 483, 'end_offset': 487}]},
    'host/SL': {'term_freq': 1,
     'tokens': [{'position': 35, 'start_offset': 155, 'end_offset': 159}]},
    'information/SL': {'term_freq': 1,
     'tokens': [{'position': 12, 'start_offset': 44, 'end_offset': 55}]},
    'pc/S

## Minimum_should_match 활용
- An optional parameter of type string that represents the minimum number of matching clauses for a document to be returned. This should only be used with “OR” operator, and is more flexible than a simple “and/or,” since users can set rules depending on the length of the phrase to be matched. 
    - **query에 있는 단어의 n%이상 존재하는 문서만 가져온다**
    - 75% (eg. 3 out of 4 words to be matched, or 6 out of 8)
    - 2  (minimum 2 words to be matched, irrespective of length of string)

min_shoud_match를 25, 75, 100 으로 변경해 보자
 - 숫자가 작아질 수록 검색 문서가 많아진다
 - 추가정보는 아래를 참고
     - [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
     - OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
         - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)

##### minimum_should_match=25

In [70]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=25
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '인터넷 뱅킹으로 예적금 해약', 'minimum_should_match': '25%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  10
# of display: 3
---------------------
_id in index:  d0f50505-15b0-486e-bda9-8f75b310220e
8.807308
ask: 인터넷 예적금 해약하려면 어떻게 해야 하나요?
Information: 인터넷에서 신규하셨고, 이후 통장발급을 받지 않으셨다면 인터넷뱅킹(http://bank.shinhan.com)의 금융상품 예금/신탁 해지 메뉴를 통해 해지하실 수 있습니다.
{'source': '신한은행', 'row': 70, 'type': '', 'timestamp': 1696832766.2919757}
---------------------
_id in index:  e21235dd-0a0d-475b-ba63-d78d4d1d63ee
5.9568224
ask: 인터넷으로 신규 예/적금 신청하는 방법을 알려주세요
Information: 인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하며 신규방법은 두 가지가 있습니다.1. 인터넷뱅킹에서 가입인터넷뱅킹 로그인을 하신 후 예금/신탁 > 신규 메뉴에서 예금 및 신탁 상품을 신규하실 수 있습니다.2. 신한S뱅크에서 가입신한S뱅크 상품센터 > 예금센터 메뉴에서 예금상품을 신규하실 수 있습니다.
{'source': '신한은행', 'row': 85, 'type': '인터넷뱅킹', 'timestamp': 1696832766.29202}
---------------------
_id in index:  c4e31eaa-b4ef-49aa-9ec4-ade3e3976859
3.7442846
ask: 신탁적금(당행)으로 이체시 토요일 인

##### minimum_should_match=75

In [74]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=75
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '인터넷 뱅킹으로 예적금 해약', 'minimum_should_match': '75%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  2
# of display: 3
---------------------
_id in index:  d0f50505-15b0-486e-bda9-8f75b310220e
8.807308
ask: 인터넷 예적금 해약하려면 어떻게 해야 하나요?
Information: 인터넷에서 신규하셨고, 이후 통장발급을 받지 않으셨다면 인터넷뱅킹(http://bank.shinhan.com)의 금융상품 예금/신탁 해지 메뉴를 통해 해지하실 수 있습니다.
{'source': '신한은행', 'row': 70, 'type': '', 'timestamp': 1696832766.2919757}
---------------------
_id in index:  e21235dd-0a0d-475b-ba63-d78d4d1d63ee
5.9568224
ask: 인터넷으로 신규 예/적금 신청하는 방법을 알려주세요
Information: 인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하며 신규방법은 두 가지가 있습니다.1. 인터넷뱅킹에서 가입인터넷뱅킹 로그인을 하신 후 예금/신탁 > 신규 메뉴에서 예금 및 신탁 상품을 신규하실 수 있습니다.2. 신한S뱅크에서 가입신한S뱅크 상품센터 > 예금센터 메뉴에서 예금상품을 신규하실 수 있습니다.
{'source': '신한은행', 'row': 85, 'type': '인터넷뱅킹', 'timestamp': 1696832766.29202}
---------------------


##### minimum_should_match=100

In [76]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=100
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '인터넷 뱅킹으로 예적금 해약', 'minimum_should_match': '100%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  1
# of display: 3
---------------------
_id in index:  d0f50505-15b0-486e-bda9-8f75b310220e
8.807308
ask: 인터넷 예적금 해약하려면 어떻게 해야 하나요?
Information: 인터넷에서 신규하셨고, 이후 통장발급을 받지 않으셨다면 인터넷뱅킹(http://bank.shinhan.com)의 금융상품 예금/신탁 해지 메뉴를 통해 해지하실 수 있습니다.
{'source': '신한은행', 'row': 70, 'type': '', 'timestamp': 1696832766.2919757}
---------------------


## Filter 활용
- document내 metadata를 활용하여 search space를 줄일 수 있다.
- 특히 filter의 경우 search 전에 수행되기 때문에, 검색 속도 향상을 기대할 수 있다
- syntax
    - filter=[{"term"**[고정]**: {"metadata.source"**[메타데이터 이름, 혹은 메타데이터 아니여도 상관없음]**: "신한은행"**[조건명]**}},]
    - list 형식으로 복수개 filter 설정 가능

In [72]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=75,
    filter=[
        {"term": {"metadata.source": "신한은행"}},
        {"term": {"metadata.type": "인터넷뱅킹"}},
    ]
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '인터넷 뱅킹으로 예적금 해약', 'minimum_should_match': '75%', 'operator': 'or'}}}], 'filter': [{'term': {'metadata.source': '신한은행'}}, {'term': {'metadata.type': '인터넷뱅킹'}}]}}}
# of searched docs:  1
# of display: 3
---------------------
_id in index:  e21235dd-0a0d-475b-ba63-d78d4d1d63ee
5.9568224
ask: 인터넷으로 신규 예/적금 신청하는 방법을 알려주세요
Information: 인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하며 신규방법은 두 가지가 있습니다.1. 인터넷뱅킹에서 가입인터넷뱅킹 로그인을 하신 후 예금/신탁 > 신규 메뉴에서 예금 및 신탁 상품을 신규하실 수 있습니다.2. 신한S뱅크에서 가입신한S뱅크 상품센터 > 예금센터 메뉴에서 예금상품을 신규하실 수 있습니다.
{'source': '신한은행', 'row': 85, 'type': '인터넷뱅킹', 'timestamp': 1696832766.29202}
---------------------


## tokenized index example (참고용)

In [47]:
test_index_info = {
    "settings": {
        "index": {
            "number_of_shards": "5",
            "knn.algo_param": {"ef_search": "512"},
            "knn": "true",
            "number_of_replicas": "2"
        },
        "analysis": {
            "tokenizer": {
                "seunjeon": {
                    "type": "seunjeon_tokenizer"
                }
            },
            "analyzer": {
                "my_analyzer": {
                    "type": "custom",
                    "tokenizer": "seunjeon"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "metadata": {
                "properties": {
                    "row": {"type": "long"},
                    "source": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "timestamp": {"type": "float"},
                    "type": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    }
                }
            },
            "text": {
                "type": "text",
                "analyzer": "my_analyzer",
                "search_analyzer": "my_analyzer",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "engine": "nmslib",
                    "space_type": "l2",
                    "name": "hnsw",
                    "parameters": {
                        "ef_construction": 512,
                        "m": 16
                    }
                }
            }
        }
    }
}